In [3]:
import hopsworks
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#ML imports
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import ensemble
from sklearn.metrics import mean_squared_error

#model upload imports
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib
import os

In [4]:
import datetime
year = datetime.date.today().year
year

2023

In [5]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/224406
Connected. Call `.close()` to terminate connection gracefully.


In [16]:
# The feature view is the input set of features for your model. The features can come from different feature groups.    
# You can select features from different feature groups and join them together to create a feature view

valuation_fg = fs.get_feature_group(name="valuationdataset", version=1)

query = valuation_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="valuationdataset_fv",
                                  version=year-2022,
                                  description="Read from valuation dataset",
                                  labels=["market_value_in_eur"],
                                  query=query)


In [17]:
valuation_fg.read()

Finished: Reading data from Hopsworks, using ArrowFlight (2.88s) 


,player_id,date,market_value_in_eur,yellow_cards,red_cards,goals,assists,minutes_played,player_name,height_in_cm,...,clean_sheets,league_es1,league_fr1,league_gb1,league_it1,league_l1,position_attack,position_defender,position_goalkeeper,position_midfield
0,423744,2023-03-16,9000000,1,0,0,2,682,Billy Gilmour,170.0,...,15,False,False,True,False,False,False,False,False,True
1,659516,2022-11-11,700000,2,0,0,0,90,Warren Bondo,177.0,...,10,False,False,False,True,False,False,False,False,True
2,256678,2023-03-21,800000,0,0,0,0,46,Filippo Romagna,186.0,...,9,False,False,False,True,False,False,True,False,False
3,95810,2022-11-03,13000000,6,0,1,9,3534,Kieran Trippier,173.0,...,15,False,False,True,False,False,False,True,False,False
4,764859,2023-03-23,4000000,1,0,0,0,130,Williot Swedberg,187.0,...,11,True,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,165007,2022-11-07,5000000,2,0,0,0,948,Denis Suárez,176.0,...,14,True,False,False,False,False,False,False,False,True
2710,306191,2023-04-06,700000,0,0,0,0,21,Jason Berthomier,177.0,...,10,False,True,False,False,False,False,False,False,True
2711,128220,2022-11-11,18000000,4,0,7,8,2914,Luis Alberto,183.0,...,23,False,False,False,True,False,False,False,False,True
2712,576121,2023-06-01,1500000,0,0,0,0,44,Josh Wilson-Esbrand,176.0,...,14,False,True,False,False,False,False,True,False,False


In [7]:
# You can read training data, randomly split into train/test sets of features (X) and labels (y)  
data = False  
while not data: 
    try:   
        length_of_df = len(valuation_fg.read())
        X_train, X_test, y_train, y_test = feature_view.train_test_split(5/length_of_df)
        #Drop date column, keeping it because might make sense later
        data = True
        
    except:
        pass

Finished: Reading data from Hopsworks, using ArrowFlight (1.30s) 
Finished: Reading data from Hopsworks, using ArrowFlight (1.40s) 


Insert new feature group that will later be used in monitoring app to measure performance

In [8]:
measurement_df = pd.concat([X_test, y_test], axis=1)

# Create a new Feature Group
feature_group = fs.get_or_create_feature_group(
    name='prediction_valuation_fg',
    version=1,
    description='Feature group containing X_test and y_test data for model performance measurement',
    primary_key=['player_id', 'date']  # Replace with your primary key column name(s)
)

# Save the merged data to the new Feature Group
feature_group.insert(measurement_df)

measurement_df

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/224406/fs/224325/fg/334164


Uploading Dataframe: 0.00% |          | Rows 0/6 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: prediction_valuation_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/224406/jobs/named/prediction_valuation_fg_1_offline_fg_materialization/executions


,player_id,date,yellow_cards,red_cards,goals,assists,minutes_played,player_name,height_in_cm,age,...,league_es1,league_fr1,league_gb1,league_it1,league_l1,position_attack,position_defender,position_goalkeeper,position_midfield,market_value_in_eur
39,212804,2023-03-27,2,0,2,0,920,Florian Tardieu,184.0,31.0,...,False,True,False,False,False,False,False,False,True,2500000
259,789082,2022-11-02,5,0,1,5,2548,Vanderson,173.0,22.0,...,False,True,False,False,False,False,True,False,False,18000000
1830,570817,2022-11-02,4,0,0,1,1042,Maxime Busi,182.0,24.0,...,False,True,False,False,False,False,True,False,False,3000000
1855,627294,2022-12-29,0,0,0,0,19,Fer Niño,191.0,23.0,...,True,False,False,False,False,True,False,False,False,3000000
2536,200512,2023-03-29,3,0,10,6,2024,Sadio Mané,174.0,31.0,...,False,False,False,False,True,True,False,False,False,45000000
2571,365172,2022-11-03,2,0,4,4,1397,Patson Daka,183.0,25.0,...,False,False,True,False,False,True,False,False,False,20000000


In [14]:
measurement_df

Index(['player_id', 'date', 'yellow_cards', 'red_cards', 'goals', 'assists',
       'minutes_played', 'player_name', 'height_in_cm', 'age', 'months_left',
       'own_goals', 'opponent_goals', 'clean_sheets', 'league_es1',
       'league_fr1', 'league_gb1', 'league_it1', 'league_l1',
       'position_attack', 'position_defender', 'position_goalkeeper',
       'position_midfield', 'market_value_in_eur'],
      dtype='object')

Dropping primary key columns

In [9]:
X_train = X_train.drop(columns=["player_id", "date", "player_name"])
X_test = X_test.drop(columns=["player_id", "date", "player_name"])

Train the model and normalize selected columns

In [10]:
from sklearn.preprocessing import StandardScaler

# Assuming X is your features DataFrame
scaler = StandardScaler()
X_train[['age', 'height_in_cm', 'minutes_played']] = scaler.fit_transform(X_train[['age', 'height_in_cm', 'minutes_played']])
X_test[['age', 'height_in_cm', 'minutes_played']] = scaler.fit_transform(X_test[['age', 'height_in_cm', 'minutes_played']])

In [11]:
# Initialize a GradientBoostingRegressor
model = GradientBoostingRegressor(learning_rate=0.15, max_depth=4, min_samples_leaf=2, min_samples_split=8, n_estimators=100)
# Fit the new GridSearchCV
model.fit(X_train, y_train)

mse = mean_squared_error(y_test, model.predict(X_test))
rmse = np.sqrt(mse)  # Calculate the RMSE
print("RMSE: ", rmse)
mean = y_train.mean()["market_value_in_eur"]
print("mean: ", mean)

RMSE:  9895485.28389804
mean:  10323384.416543575


# upload to hopsworks

In [12]:
# We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
mr = project.get_model_registry()

# The contents of the 'iris_model' directory will be saved to the model registry. Create the dir, first.
model_dir = f"valuation_model_v{year-1}_{year}"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)
    

# Save your model
joblib.dump(model, model_dir + "/valuation_model.pkl")
joblib.dump(scaler, model_dir + "/scaler.pkl")

# Specify the schema of the model's input/output using the features (X_train) and labels (y_train)
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

# Create an entry in the model registry that includes the model's name, desc, metrics
valuation_model = mr.python.create_model(
    name=model_dir, 
    metrics={"RMSE" : rmse, "Mean": mean},
    model_schema=model_schema,
    description=f"Valuation model based on data from the season {year-1}/{year}"
)

# Upload the model to the model registry, including all files in 'model_dir'
valuation_model.save(model_dir)

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/1023 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/283580 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1635 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/224406/models/valuation_model_v2022_2023/3


Model(name: 'valuation_model_v2022_2023', version: 3)